In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import numpy as np
import pandas as pd
import string
from nltk import word_tokenize
import re
punct = string.punctuation
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_csv('/Users/gauneg/datasets/fun_dsets/sourth_park_ds.csv')

In [3]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


CONTEXT_SIZE = 2
EMBEDDINGS_DIM = 10

In [4]:
def context_ngrams(word_arr, context_size, mode="uni_dir"):
    txt = []
    end_point = len(word_arr) - context_size if mode=="bi_dir" else len(word_arr)
    for i in range(context_size, end_point):
        if mode=="uni_dir":
            txt.append([word_arr[i-context_size: i], word_arr[i]])
        else:
            txt.append([word_arr[i-context_size: i] +  word_arr[i+1: i + context_size+1], 
                        word_arr[i]])
    return txt


In [5]:
vocab = set(test_sentence)
word_to_ix = {word:i for i, word in enumerate(vocab)}

ngrams = context_ngrams(test_sentence, CONTEXT_SIZE)

In [6]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size) -> None:
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.liner1 = nn.Linear(context_size * embedding_dim, 128, )
        self.liner2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        # print('Embedding output', embeds.shape)\
        out = F.relu(self.liner1(embeds))
        
        out = self.liner2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
        

In [7]:
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDINGS_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(100):
    total_loss = 0
    for context, target in ngrams:
        context_ids = torch.tensor(list(map(lambda x: word_to_ix[x], context)), dtype=torch.long)
        lab_ids = torch.tensor([word_to_ix[target]], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_ids)
        loss = loss_function(log_probs, lab_ids)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(total_loss)
        

In [9]:
PRED_INDEX = 14
ix_to_word = {v:k for k,v in word_to_ix.items()}
with torch.no_grad():

    for i, (context, target) in enumerate(ngrams):
        if i==PRED_INDEX:
            context_ids = torch.tensor(list(map(lambda x: word_to_ix[x], context)), dtype=torch.long)
            lab_ids = torch.tensor([word_to_ix[target]], dtype=torch.long)
            y_pred = model(context_ids)
            pred_index = y_pred.argmax(dim=-1)
            print(f"context:{context}, target:{target}, pred:{ix_to_word[pred_index[0].item()]}")


context:['field,', 'Thy'], target:youth's, pred:youth's
